# Metadata json to netCDF converter
Converting a jason metadata template file to a comtomized netCDF file, which contains only the header without data. 
From the template, users could add the attributes and change the value for each attribute for dimensions, global attributes and variables. 

In [29]:
# Import modules
import os 
import json
from netCDF4 import Dataset

In [30]:
# Define output nc/json filename
#output_file = "point_mooring"
#output_file = "profile_ctd"
#output_file = "profile_sonde"
#output_file = "satellite_l2"
#output_file = "satellite_L3_L4"
output_file = "trajectory_drifter"

In [31]:
# Listing all the template JSON files in smpl/ sub-directory
parentdir = os.getcwd()
jpath = os.path.join(parentdir, 'smpl') 
smpl_dir_list = os.listdir(jpath)
x=[print(i,smpl) for i, smpl in enumerate(smpl_dir_list)]

0 satellite_l2_template.json
1 satellite_L3_L4_template.json
2 profile_sonde_template.json
3 trajectory_drifter_template.json
4 profile_ctd_template.json
5 point_mooring_template.json
6 meta_tmpl.json


In [32]:
# Choose the template JSON file from the list by entering the index value.
ind = input("Choose the sample_json file by entering the index value: ")
# example_json = smpl_dir_list[int(ind)]
smpl_json = os.path.join(jpath,smpl_dir_list[int(ind)])
print(smpl_json)

Choose the sample_json file by entering the index value:  3


/Users/earmstro/Downloads/metadata_templates-main/notebook/smpl/trajectory_drifter_template.json


In [33]:
# Read the json file
with open(smpl_json, 'r') as f:
    mData = json.loads(f.read())

In [34]:
# Print the sample json file
json_formatted_str = json.dumps(mData, indent=2)
print(json_formatted_str)

{
  "dimensions": {
    "time": 54
  },
  "variables": {
    "time": {
      "type": "double",
      "dimensions": [
        "time"
      ],
      "attributes": {
        "long_name": "Time of Salinity Drifter measurement",
        "axis": "T",
        "standard_name": "time",
        "units": "days since 1950-01-01T00:00:00",
        "coverage_content_type": "coordinate"
      }
    },
    "longitude": {
      "type": "double",
      "dimensions": [
        "time"
      ],
      "attributes": {
        "_FillValue": -9999,
        "long_name": "Longitude of Salinity Drifter measurement",
        "valid_max": 180,
        "valid_min": -180,
        "axis": "X",
        "standard_name": "longitude",
        "units": "degrees_east",
        "coverage_content_type": "coordinate"
      }
    },
    "latitude": {
      "type": "double",
      "dimensions": [
        "time"
      ],
      "attributes": {
        "_FillValue": -9999,
        "long_name": "Latitude of Salinity Drifter measurem

In [75]:
# function to add and remove key:value pair
def add_rem_attri(grp_dict, isValGrp = ''):
    # Add the new attribute
    newKey = input("++ Add a new attribute KEY? (KEY-Name/Enter(no)): ")
    while newKey != '':
        if isValGrp == 'var':
            newValtmpl = input("Enter the template(example) var name: ")
            grp_dict[newKey]= grp_dict[newValtmpl]
        else:
            grp_dict[newKey]= ''
            
        newKey = input("++ Add another new attribute KEY? (KEY-Name/Enter(no)): ")
            
    # Remove the example_var variables
    delKey = input("-- Delete a attribute KEY? (KEY-Name/Enter(no)): ")
    while delKey != '':
        if delKey in grp_dict.keys(): 
            del grp_dict[delKey]

        delKey = input("-- Delete another KEY? (KEY-Name/Enter(no)): ")
        
    print([key for key in grp_dict.keys()])


In [36]:
# A recursive function to change the attribute values.
def change_val(grp_attr):
    for key, value in grp_attr.items():
        if (type(value) is dict):
            #print("\n", "GROUP NAME: " + key)
            x = input("#### Change the attributes of '" + key + "'? (Y/Enter(no)): ")
            if x.lower() == 'y':
                # Print the json file
                json_formatted_str = json.dumps(value, indent=2)
                print(json_formatted_str)
                change_val(value)
        else:
            #print ('\t', key, ' = ', value)
            x = input("--> Change the '" + key + "' value? (Y/Enter(no)): ")
            if x.lower() == 'y':
                type_str = input("  -->> Enter the data type (int, float, list, or default('str')): ")
                if type_str ==  "int":
                    grp_attr[key] = int(input("Enter new value for '" + key + "': "))
                elif type_str ==  "float":
                    grp_attr[key] = float(input("Enter new value for '" + key + "': "))
                elif type_str ==  "list":
                    list_type = input("  -->> Enter the LIST type (int, float, or default('str')): ")
                    strList = input("Enter new value for '" + key + "': ").split(',')
                    if list_type ==  "int":
                        grp_attr[key] = list(map(int,strList))
                    elif list_type ==  "float":
                        grp_attr[key] = list(map(float,strList))
                    else:
                        grp_attr[key] = strList
                else:
                    grp_attr[key] = input("Enter new value for '" + key + "': ")

# Review and change "dimensions"

In [37]:
# dimensions fields
dims = mData['dimensions']
dims

{'time': 54}

In [38]:
# Add and/or remove Key:Value pairs of dimensions
add_rem_attri(dims)

++ Add a new attribute KEY? (KEY-Name/Enter(no)):  depth
++ Add another new attribute KEY? (KEY-Name/Enter(no)):  latitude
++ Add another new attribute KEY? (KEY-Name/Enter(no)):  longitude
++ Add another new attribute KEY? (KEY-Name/Enter(no)):  
-- Delete a attribute KEY? (KEY-Name/Enter(no)):  


['time', 'depth', 'latitude', 'longitude']


In [56]:
# Review and change the attribute values 
change_val(dims)

--> Change the 'time' value? (Y/Enter(no)):  
--> Change the 'depth' value? (Y/Enter(no)):  
--> Change the 'latitude' value? (Y/Enter(no)):  y
  -->> Enter the data type (int, float, list, or default('str')):  int
Enter new value for 'latitude':  100
--> Change the 'longitude' value? (Y/Enter(no)):  y
  -->> Enter the data type (int, float, list, or default('str')):  int
Enter new value for 'longitude':  200


In [57]:
print(dims)

{'time': 60, 'depth': 10, 'latitude': 100, 'longitude': 200}


# Review and change "global attributes"

In [41]:
# Global attribute dict
glb_attr = mData['global_attributes']
glb_attr

{'Conventions': 'CF-1.6, ACDD 1.3',
 'Metadata_Conventions': 'Unidata Dataset Discovery v1.3',
 'featureType': 'trajectory',
 'cdm_data_type': 'Trajectory',
 'nodc_template_version': 'NODC_NetCDF_trajectory_Template_v2.0',
 'standard_name_vocabulary': 'CF Standard Name Table v27',
 'title': '',
 'summary': '',
 'source': '',
 'platform': 'Drifter',
 'instrument': '',
 'uuid': '',
 'id': 'COLLECTION_SHORTNAME',
 'metadata_link': '',
 'references': '',
 'sea_name': '',
 'naming_authority': '',
 'time_coverage_start': 'YYYY-MM-DDT00:00:00Z',
 'time_coverage_end': 'YYYY-MM-DDT00:00:00Z',
 'time_coverage_resolution': 'PT1S',
 'geospatial_lat_min': -90,
 'geospatial_lat_max': 90,
 'geospatial_lat_units': 'degrees_north',
 'geospatial_lat_resolution': '0.1 degree',
 'geospatial_lon_min': -180,
 'geospatial_lon_max': 180,
 'geospatial_lon_units': 'degrees_east',
 'geospatial_lon_resolution': '0.1 degree',
 'institution': '',
 'publisher_name': 'Physical Oceanography Distributed Active Archive 

In [42]:
# Add and/or remove Key:Value pairs of dimensions
add_rem_attri(glb_attr)

++ Add a new attribute KEY? (KEY-Name/Enter(no)):  cruise_id
++ Add another new attribute KEY? (KEY-Name/Enter(no)):  
-- Delete a attribute KEY? (KEY-Name/Enter(no)):  


['Conventions', 'Metadata_Conventions', 'featureType', 'cdm_data_type', 'nodc_template_version', 'standard_name_vocabulary', 'title', 'summary', 'source', 'platform', 'instrument', 'uuid', 'id', 'metadata_link', 'references', 'sea_name', 'naming_authority', 'time_coverage_start', 'time_coverage_end', 'time_coverage_resolution', 'geospatial_lat_min', 'geospatial_lat_max', 'geospatial_lat_units', 'geospatial_lat_resolution', 'geospatial_lon_min', 'geospatial_lon_max', 'geospatial_lon_units', 'geospatial_lon_resolution', 'institution', 'publisher_name', 'publisher_email', 'publisher_url', 'publisher_type', 'publisher_institution', 'creator_type', 'creator_institution', 'creator_email', 'creator_name', 'creator_role', 'project', 'processing_level', 'keywords_vocabulary', 'keywords', 'acknowledgement', 'date_created', 'date_modified', 'date_issued', 'date_metadata_modified', 'program', 'product_version', 'license', 'comment', 'cruise_id']


In [ ]:
# Loop through the global attributes, change the attribute value as needed.
change_val(glb_attr)

In [43]:
glb_attr

{'Conventions': 'CF-1.6, ACDD 1.3',
 'Metadata_Conventions': 'Unidata Dataset Discovery v1.3',
 'featureType': 'trajectory',
 'cdm_data_type': 'Trajectory',
 'nodc_template_version': 'NODC_NetCDF_trajectory_Template_v2.0',
 'standard_name_vocabulary': 'CF Standard Name Table v27',
 'title': '',
 'summary': '',
 'source': '',
 'platform': 'Drifter',
 'instrument': '',
 'uuid': '',
 'id': 'COLLECTION_SHORTNAME',
 'metadata_link': '',
 'references': '',
 'sea_name': '',
 'naming_authority': '',
 'time_coverage_start': 'YYYY-MM-DDT00:00:00Z',
 'time_coverage_end': 'YYYY-MM-DDT00:00:00Z',
 'time_coverage_resolution': 'PT1S',
 'geospatial_lat_min': -90,
 'geospatial_lat_max': 90,
 'geospatial_lat_units': 'degrees_north',
 'geospatial_lat_resolution': '0.1 degree',
 'geospatial_lon_min': -180,
 'geospatial_lon_max': 180,
 'geospatial_lon_units': 'degrees_east',
 'geospatial_lon_resolution': '0.1 degree',
 'institution': '',
 'publisher_name': 'Physical Oceanography Distributed Active Archive 

# Review and change "variables"

In [72]:
# Variables group
var_attr = mData['variables']
#var_attr
print([key for key in var_attr.keys()])

['time', 'longitude', 'latitude', 'example_temperature_at_18cm', 'example_quality_flag', 'example_id', 'temperature_at_1m']


In [74]:
# Add and/or remove Key:Value for variables
add_rem_attri(var_attr, 'var')

++ Add a new attribute KEY? (KEY-Name/Enter(no)):  test
Enter the template(example) var name:  example_id
++ Add another new attribute KEY? (KEY-Name/Enter(no)):  
-- Delete a attribute KEY? (KEY-Name/Enter(no)):  


['time', 'longitude', 'latitude', 'example_temperature_at_18cm', 'example_quality_flag', 'example_id', 'temperature_at_1m', 'test']


In [70]:
# Loop through the variables
change_val(var_attr)


#### Change the attributes of 'time'? (Y/Enter(no)):  
#### Change the attributes of 'longitude'? (Y/Enter(no)):  
#### Change the attributes of 'latitude'? (Y/Enter(no)):  
#### Change the attributes of 'example_temperature_at_18cm'? (Y/Enter(no)):  
#### Change the attributes of 'example_quality_flag'? (Y/Enter(no)):  
#### Change the attributes of 'example_id'? (Y/Enter(no)):  y


{
  "type": "double",
  "dimensions": [
    "time"
  ],
  "attributes": {
    "long_name": "Drifter ID",
    "standard_name": "drifter_id",
    "coverage_content_type": "referenceInformation"
  }
}


--> Change the 'type' value? (Y/Enter(no)):  
--> Change the 'dimensions' value? (Y/Enter(no)):  
#### Change the attributes of 'attributes'? (Y/Enter(no)):  y


{
  "long_name": "Drifter ID",
  "standard_name": "drifter_id",
  "coverage_content_type": "referenceInformation"
}


--> Change the 'long_name' value? (Y/Enter(no)):  
--> Change the 'standard_name' value? (Y/Enter(no)):  
--> Change the 'coverage_content_type' value? (Y/Enter(no)):  
#### Change the attributes of 'temperature_at_1m'? (Y/Enter(no)):  


In [71]:
# reviewing all the variables
var_attr

{'time': {'type': 'double',
  'dimensions': ['time'],
  'attributes': {'long_name': 'Time of Salinity Drifter measurement',
   'axis': 'T',
   'standard_name': 'time',
   'units': 'days since 1950-01-01T00:00:00',
   'coverage_content_type': 'coordinate'}},
 'longitude': {'type': 'double',
  'dimensions': ['time'],
  'attributes': {'long_name': 'Longitude of Salinity Drifter measurement',
   'valid_max': 180,
   'valid_min': -180,
   'axis': 'X',
   'standard_name': 'longitude',
   'units': 'degrees_east',
   'coverage_content_type': 'coordinate'}},
 'latitude': {'type': 'double',
  'dimensions': ['time'],
  'attributes': {'long_name': 'Latitude of Salinity Drifter measurement',
   'valid_max': 90,
   'valid_min': -90,
   'axis': 'Y',
   'standard_name': 'latitude',
   'units': 'degrees_north',
   'coverage_content_type': 'coordinate'}},
 'example_temperature_at_18cm': {'type': 'double',
  'dimensions': ['time'],
  'attributes': {'long_name': 'Sea water temperature at 18cm',
   'valid_

# Write the new metadata to netCDF and json files to out_dir directory

In [58]:
# Define the output netCDF file path
oPath = os.path.join(parentdir, 'out_dir') 
empty_nc = os.path.join(oPath, output_file + '.nc')
empty_nc

'/Users/earmstro/Downloads/metadata_templates-main/notebook/out_dir/trajectory_drifter.nc'

In [59]:
if not os.path.exists(oPath):
    os.makedirs(oPath)

In [68]:
# Create an empty netCDF file
with Dataset(empty_nc, "w") as nc:
    # Add dimensions
    for dim_name, dim_size in mData["dimensions"].items():
        nc.createDimension(dim_name, dim_size)
        
    # Add variables and their metadata
    for var_name, var_attrs in mData["variables"].items():
     fill_value = var_attrs["attributes"].pop("_FillValue", None)  # remove it before setattr()
     var = nc.createVariable(
        var_name,
        var_attrs["type"],
        var_attrs["dimensions"],
        fill_value=fill_value)
     for attr_name, attr_value in var_attrs["attributes"].items():
       setattr(var, attr_name, attr_value)

    # Add global attributes
    for global_attr_name, global_attr_value in mData["global_attributes"].items():
        setattr(nc, global_attr_name, global_attr_value)


print(f"Empty netCDF file '{empty_nc}' created successfully with embedded JSON metadata attributes.")

Empty netCDF file '/Users/earmstro/Downloads/metadata_templates-main/notebook/out_dir/trajectory_drifter.nc' created successfully with embedded JSON metadata attributes.


In [61]:
# Output the modified metadata dict to a json file.
out_json = os.path.join(oPath, output_file + '.json')
with open(out_json, 'w', encoding='utf-8') as f:
    json.dump(mData, f, ensure_ascii=False, indent=4)

In [69]:
# system call to display the customized meta info from the netCDF. 
!ncdump -h {empty_nc}

netcdf trajectory_drifter {
dimensions:
	time = 60 ;
	depth = 10 ;
	latitude = 100 ;
	longitude = 200 ;
variables:
	double time(time) ;
		time:long_name = "Time of Salinity Drifter measurement" ;
		time:axis = "T" ;
		time:standard_name = "time" ;
		time:units = "days since 1950-01-01T00:00:00" ;
		time:coverage_content_type = "coordinate" ;
	double longitude(time) ;
		longitude:long_name = "Longitude of Salinity Drifter measurement" ;
		longitude:valid_max = 180. ;
		longitude:valid_min = -180. ;
		longitude:axis = "X" ;
		longitude:standard_name = "longitude" ;
		longitude:units = "degrees_east" ;
		longitude:coverage_content_type = "coordinate" ;
	double latitude(time) ;
		latitude:long_name = "Latitude of Salinity Drifter measurement" ;
		latitude:valid_max = 90. ;
		latitude:valid_min = -90. ;
		latitude:axis = "Y" ;
		latitude:standard_name = "latitude" ;
		latitude:units = "degrees_north" ;
		latitude:coverage_content_type = "coordinate" ;
	double example_temperature_at_18cm(tim